In [21]:
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams.update(plt.rcParamsDefault)
from sklearn.manifold import MDS
import math
import scipy.io
import numpy as np

In [31]:
with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_all_EPSP_young.pkl', 'rb') as file:
    All_dy_list_young = pickle.load(file)
with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_all_noise_corr.pkl', 'rb') as file:
    All_noise_corr = pickle.load(file)

with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_signal_corr_WT', 'rb') as file:
    All_signal_corr_WT = pickle.load(file)

with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_signal_corr_AD', 'rb') as file:
    All_signal_corr_AD = pickle.load(file)

In [34]:
#All_markov_result = []

import networkx as nx
def Connector(Q):
    D = nx.to_networkx_graph(Q, create_using=nx.DiGraph())
    Isolate_list = list(nx.isolates(D))
    if len(Isolate_list) > 0:
        for i in Isolate_list:
            if i == 0:
                Q[i + 1, i] = 0.0001
            else:
                Q[i - 1, i] = 0.0001
    del D
    return Q


type = 'AD'
All_markov_signal = []

if type == 'AD':
    Signal_corr = All_signal_corr_AD
elif type == 'WT':
    Signal_corr = All_signal_corr_WT
    
for index in range(len(Signal_corr)):
    
    A = Signal_corr[index]
    N = A.shape[0]
    np.fill_diagonal(A, 0)
    # print(max(A[:,4]))
    # A=np.where(A > 0.09, 1, 0)
    k = 10
    
    # W.sort(reverse=True)
    B1 = np.zeros((N, N))
    for i in range(N):
        W = sorted(A[i, :], reverse=True)
        #     print( W[k])
        B1[i, :] = np.where(A[i, :] > W[k], 1, 0)
    
    # B=np.multiply(B1,A)
    # print(W[k])
    # print(A[20,1:20])
    # print(B[20,1:20])
    
    C1 = np.zeros((N, N))
    for i in range(N):
        W = sorted(A[:, i], reverse=True)
        #     print( W[k])
        C1[:, i] = np.where(A[:, i] > W[k], 1, 0)
    # C=np.multiply(C1,A)
    Q1 = B1 + C1
    Q2 = np.where(Q1 > .9, 1, 0)
    
    Q = np.multiply(Q2, A)
    # del A
    del B1, C1, Q1, Q2
    for i in range(Q.shape[0]):
        # 检查该行是否全为零
        if np.all(Q[i] == 0):
            # 如果是全为零，随机选择一个元素，并将其赋值为 0.001
            random_index = np.random.randint(0, Q.shape[1])  # 随机选择一个列索引
            Q[i, random_index] = 0.001
    import pygenstability as pgs
    import scipy.sparse as sp
    
    adjacency = sp.csr_matrix(Q)
    test = sp.csgraph.connected_components(adjacency)
    all_results = pgs.run(adjacency,
                          min_scale=-1.5,
                          max_scale=1.5,
                          n_scale=200,
                          n_tries=600,
                          constructor="directed")
    
    all_results = pgs.identify_optimal_scales(all_results, kernel_size=14, window_size=14)
    All_markov_signal.append(all_results)
    print('Finished No.' + f'{index}_markov_result')
    
with open('/Users/sonmjack/Downloads/age2 result_fam1/fam1_all_markov_signal_'+f'{type}.pkl', 'wb') as file:
    pickle.dump(All_markov_signal, file)

  8%|▊         | 16/200 [00:18<03:29,  1.14s/it]
Process SpawnPoolWorker-161:
Traceback (most recent call last):
  File "/Users/sonmjack/miniconda/envs/pygneuron/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/sonmjack/miniconda/envs/pygneuron/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/sonmjack/miniconda/envs/pygneuron/lib/python3.8/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/Users/sonmjack/miniconda/envs/pygneuron/lib/python3.8/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/Users/sonmjack/miniconda/envs/pygneuron/lib/python3.8/site-packages/pygenstability/pygenstability.py", line 57, in wrap
    result = f(*args, **kw)
  File "/Users/sonmjack/miniconda/envs/pygneuron/lib/python3.8/site-packages/pygenstability/pygenstability.py", line 333, in _optimise
    stability, communi

KeyboardInterrupt: 

In [28]:
from pygenstability import plotting
_ = plotting.plot_scan(all_results,figure_name='scan_results.pdf', use_plotly=True, live=True, plotly_filename='scan_results.html')